In [ ]:
"""
Script que lê um arquivo csv, com uma coluna nomeada "name" onde constam nomes de espécies/generos
de organismos, e retorna as familias dos respectivos taxons

O script apresenta alguns problemas ao identificar taxons sem classificação definida, ficar atento
a isso.
"""

from Bio import Entrez, SeqIO
import pandas as pd

# Cria função obter_familia que recebe um genero ou espécie e retorna a familia
def obter_familia(taxon):
    #Extrai o primeiro termo, que é o genero
    try:
        genero = taxon.split()[0] 
    
        #Nescessário adicionar seu email para acessar o ncbi
        Entrez.email = "danielpolita8@gmail.com" 

        #Realiza a pesquica no ncbi a partir do nome da espécie/genero.
        #O termo [Organism] serve para que a pesquisa fique restrita a organismos
        #A função Entrez.esearch() retorna um identificador(ID) do registro correspondente
        # a espécie/genero pesquisado. Esse ID é usado para obter informações sobre a espécie/genero.
        handle = Entrez.esearch(db="taxonomy", term=genero + "[Organism]")
        record = Entrez.read(handle)
        handle.close()
        taxid = record["IdList"][0]

        #Recupera informações apartir do ID obtido na etapa anterior
        #A função Entrez.efetch recupera essas informações no formato XML
        #A função Entrez.read() lê as informações obtidas no formato XML
        handle = Entrez.efetch(db="taxonomy", id=taxid, retmode="xml")
        record = Entrez.read(handle)
        handle.close()

        #Itera sobre os dados recuperados para encontrar a familia corespondente ao genero
        #A propriedade 'LineageEx' contém informações sobre a classificação taxonômica do organismo.
        #Dentro do loop, verificamos o campo Rank para encontrar a entrada que corresponde à família.
        #Quando identifica a familia correspondente o nome cientifico da familia é retornado.
        for data in record:
            for info in data["LineageEx"]:
                if info["Rank"] == "family":
                    return info["ScientificName"]
            
    except Exception as e:
        print("erro", e)
        return "Desconhecida"
    
#Carrega o arquivo csv com as espécies/genero
df = pd.read_csv("Fungi9_names.CSV")

#Adiciona uma coluna familia ao Dataframe e preenche a coluna com as informações sobre as familias
df["familia"] = df["name"].apply(obter_familia)

#Salvar o Dataframe atualizo em um novo arquivo csv
df.to_csv("Fungi9_Taxons_com_familias.csv", index=False)

#Mostra o dataframe atualizado
print(df)
